In [ ]:
import geopandas as gpd
import rioxarray
import rasterstats as rs
import numpy as np
import xarray as xr

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

### Exploring Land Surface Temperature

ESA Land Surface Temperature Climate Change Initiative (LST_cci): Monthly Multisensor Infra-Red (IR) Low Earth Orbit (LEO) land surface temperature (LST) time series level 3 supercollated (L3S) global product (1995-2020), version 2.00. More information from the CEDA Archive can be found [here](https://catalogue.ceda.ac.uk/uuid/785ef9d3965442669bff899540747e28).

Now let's explore Land Surface Temperature, but this time let's try to access directly from the URL

In [ ]:
# must add `#mode=bytes` to the end (see: https://github.com/Unidata/netcdf4-python/issues/1043)
# url = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/12/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201201000000-fv2.00.nc#mode=bytes"
url = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/11/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201101000000-fv2.00.nc#mode=bytes"
# can check status of CEDA core archives here: https://stats.uptimerobot.com/vZPgQt7YnO, currently `dap` is down

In [ ]:
ds_disk = xr.open_dataset(url)
ds_disk

Before we do any processing of the data, we will want to clip the global dataset down to our AOI (continent of Africa) to reduce the size and processing time. To do this, we'll import the national administrative boundaries (admin 0) for Africa in order to create a bounding box for the continent.

In [ ]:
# import geopandas as gpd

# admin0_gdf = gpd.read_file(
#     "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_0&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
# )
# # admin0_gdf

# print(admin0_gdf.crs)

In [ ]:
# xmin, ymin, xmax, ymax = admin0_gdf.geometry.total_bounds
# print(xmin, ymin, xmax, ymax)

In [ ]:
# sorting solves CRS issue? see: https://gis.stackexchange.com/questions/396365/using-rioxarray-to-assign-spatial-reference-epsg4326-to-netcdf-built-from-csv
ds_sort = ds_disk.sortby(["time", "lat", "lon"])

In [ ]:
# Assign crs to match boundaries
ds_sort.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
# Subset to bounding box of African continent
lst_clip = ds_sort["lst"].rio.clip_box(
    minx=-25.35874748,
    miny=-46.9813795,
    maxx=63.5026492,
    maxy=37.560954,
    crs="EPSG:4326",
)

In [ ]:
lst_clip

In [ ]:
lst_clip.coords

In [ ]:
# Convert from Kelvin to Celsius
lst_africa_c = lst_clip - 273.15
lst_africa_c

We'll visualize only 1 time step here - but it would be great to visualize the whole time series:
* we could visualize instead as a line graph
* or as visualization that loops over the entire time period 

But more importantly, we're going to have to aggregate these daily values to monthly. 

In [ ]:
lst_clip.dims
lst_clip.coords

In [ ]:
# selecting single time point (first time-step)
lst_2d = lst_clip.squeeze()
lst_2d

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.contourf(lst_2d, 20, cmap=plt.get_cmap("coolwarm"))
plt.colorbar()
plt.show()

In [ ]:
admin2_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_2&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
admin2_gdf
print(admin2_gdf.crs)

In [ ]:
admin2_gdf.boundary.plot()

In [ ]:
import pandas as pd
from rasterstats import zonal_stats

In [ ]:
lst_2d.values

In [ ]:
# good affine
affine = lst_2d.rio.transform()
gdal = list(affine.to_gdal())
gdal[-1] = -gdal[-1]
affine = affine.from_gdal(*gdal)
affine

In [ ]:
admin2_lst = rs.zonal_stats(
    admin2_gdf,
    lst_2d.values,
    affine=affine,  # using corrected affine above
    stats="mean",
    nodata=-999,  # eliminate "nodata" warnings
)
admin2_lst

In [ ]:
# Plot raster and shapefile
fig, ax = plt.subplots(figsize=(10, 10))

# plot raster LST
plt.contourf(lst_2d, 20, cmap=plt.get_cmap("coolwarm"))
plt.colorbar()
plt.title("LST in African Districts")

# plot boundaries
admin2_gdf.plot(edgecolor="black", lw=0.7)

plt.show()
plt.tight_layout()